
$\qquad$ $\qquad$ $\qquad$  **TDA206/DIT206 Discrete Optimization: Home Assignment 2 -- Integer LP and Relaxation** <br />
$\qquad$ $\qquad$ $\qquad$                   **Grader: Marc Constantin** <br />
$\qquad$ $\qquad$ $\qquad$                   **Due Date: 17th Feb** <br />
$\qquad$ $\qquad$ $\qquad$                   **Submitted by: Rikard Roos, 941003-9292, rikardro@student.chalmers.se** <br />
$\qquad$ $\qquad$ $\qquad$                   **Submitted by: Olof Lindberg, 020725-0192, ololin0725@gmail.com** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.
(5 points) 
There are 4 space colonies, each of which  requires a certain number of plasma conduits. There are 3 starbases in the vicinity. Each of them has total number of conduits they can spare and supply to the colonies. For each pair of starbase and colony, there is an associated cost for sending a cargo ship  (each of which carries one plasma conduit), as shown in the table below:


\begin{array}{l|c|c|c|c|c} 
      & Triacus & New Berlin  & Strnad  & Vega  & supply\\ \hline
 Farpoint &   6 &  9 & 10 & 8 & 35\\
 Yorktown &  9 & 5 & 16 & 14 & 40\\
 Earhart & 12 &  7 & 13 & 9 & 50\\ \hline
    demand & 20 &30&30&45& \left(\sum=125\right) \\ 
\end{array}

Your goal is to supply the colonies the plasma conduits they need, at minimum cost.


Formulate a LP to solve the problem, explain why the solution is integral with a proof.

In [31]:
import cvxpy as cp 
import numpy as np

# Variables
x = cp.Variable(12, nonneg=True)

# Objective function
c = [6, 9, 10, 8, 9, 5, 16, 14, 12, 7, 13, 9]
obj = cp.Minimize(c @ x)

# Constraints
A = np.array([[1,1,1,1,0,0,0,0,0,0,0,0],
              [0,0,0,0,1,1,1,1,0,0,0,0],
              [0,0,0,0,0,0,0,0,1,1,1,1],
              [-1,0,0,0,-1,0,0,0,-1,0,0,0],
              [0,-1,0,0,0,-1,0,0,0,-1,0,0],
              [0,0,-1,0,0,0,-1,0,0,0,-1,0],
              [0,0,0,-1,0,0,0,-1,0,0,0,-1]])

Ax = A @ x
b = [35, 40, 50, -20, -30, -30, -45]

constraints = [Ax <= b]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Objective value: 1020.0
Solution vector: [10.0, 0.0, 25.0, 0.0, 10.0, 30.0, 0.0, 0.0, 0.0, 0.0, 5.0, 45.0]


The problem consists of minimizing the cost of transporting goods to colonies under the supply and demand constraints. If we express our LP on $\mathbf{A} \mathbf{x} \leq \mathbf{b}$ form, we can use the Hoffman-Kruskal Theorem to prove that the solution space consists of integers.

We define a LP problem as follows:

$$
\begin{aligned}
    &\text{Minimize} \quad && \mathbf{c}^T \mathbf{x} \\
    &\text{subject to} \quad && \mathbf{A} \mathbf{x} \leq \mathbf{b}, \\
    & && \mathbf{x} \geq 0.
\end{aligned}
$$

where $\mathbf{x}$ is the variable vector, $\mathbf{c}$ is the cost vector, $\mathbf{A}$ is the constraints matrix and $\mathbf{b}$ is the constraints value vector.

Since the demand constraints are on the form $\mathbf{A} \mathbf{x}_i \geq \mathbf{b}_i$, we define them as $-\mathbf{A} \mathbf{x}_i \leq -\mathbf{b}_i$. Thus, we get the following inequalities:

$$
\begin{bmatrix}
x_1 + x_2 + x_3 + x_4 \\
x_5 + x_6 + x_7 + x_8 \\
x_9 + x_{10} + x_{11} + x_{12} \\
-x_1 - x_5 - x_9 \\
-x_2 - x_6 - x_{10} \\
-x_3 - x_7 - x_{11} \\
-x_4 - x_8 - x_{12} \\
\end{bmatrix}
\leq
\begin{bmatrix}
35 \\
40 \\
50 \\
-20 \\
-30 \\
-30 \\
-45 \\
\end{bmatrix}

$$

From this, the constraints matrix $\mathbf{A}$ can be computed:

$$
\mathbf{A}\mathbf{x} =
\begin{bmatrix}
1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1\\
-1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0\\
0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0\\
0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0\\
0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1\\
\end{bmatrix}

\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8 \\
x_9 \\
x_{10} \\
x_{11} \\
x_{12} \\
\end{bmatrix}
=
\begin{bmatrix}
x_1 + x_2 + x_3 + x_4 \\
x_5 + x_6 + x_7 + x_8 \\
x_9 + x_{10} + x_{11} + x_{12} \\
-x_1 - x_5 - x_9 \\
-x_2 - x_6 - x_{10} \\
-x_3 - x_7 - x_{11} \\
-x_4 - x_8 - x_{12} \\
\end{bmatrix}
\leq
\begin{bmatrix}
35 \\
40 \\
50 \\
-20 \\
-30 \\
-30 \\
-45 \\
\end{bmatrix}
= \mathbf{b}
$$

According to the Hoffman-Kruskal Theorem, a region $\{x : \mathbf{A} \mathbf{x} = \mathbf{b}, \mathbf{x} \geq 0\}$ or $\{x : \mathbf{A} \mathbf{x} \leq \mathbf{b}, \mathbf{x} \geq 0\}$ is integral if $\mathbf{A}$ is a totally unimodular matrix (TUM) and $\mathbf{b}$ is integral. Since $\mathbf{b}$ is a vector of integers, we only have to show that $\mathbf{A}$ is a TUM.

A matrix $\mathbf{A}$ is totally unimodular if:

- entries $\in {-1,0,1}$
- in each column , there are at most 2 non-zero entries
- rows can be partitioned into two sets s.t. each column:
    - if non-zero entries have opposite signs, they are in the same set
    - if non-zero entries have the same signs, thay are in different sets

Since $\mathbf{A}$ meet all requirments $\mathbf{A}$ is a TUM, and thus the solution space is integral.

To prove that the optimal solution is integral we will use the Hoffman-Krusal Theorem.

Let's formulate our problem with the following program:

$$
\begin{aligned}
    &\text{Minimize} \quad && \mathbf{c}^T \mathbf{x} \\
    &\text{subject to} \quad && A\mathbf{x} \leq \mathbf{b}, \\
    & && \mathbf{x} \geq 0.
\end{aligned}
$$


$ A\mathbf{x} \leq \mathbf{b}$ can be written as follows:

$$
\begin{bmatrix}
x_1 + x_2 + x_3 + x_4 \\
x_5 + x_6 + x_7 + x_8 \\
x_9 + x_{10} + x_{11} + x_{12} \\
-x_1 - x_5 - x_9 \\
-x_2 - x_6 - x_{10} \\
-x_3 - x_7 - x_{11} \\
-x_4 - x_8 - x_{12} \\
\end{bmatrix}
\leq
\begin{bmatrix}
35 \\
40 \\
50 \\
-20 \\
-30 \\
-30 \\
-45 \\
\end{bmatrix}

$$

From this matrix A can be computed:

$$
\begin{bmatrix}
1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1\\
-1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0\\
0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0\\
0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0\\
0 & 0 & 0 & -1 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & -1\\
\end{bmatrix}

\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8 \\
x_9 \\
x_{10} \\
x_{11} \\
x_{12} \\
\end{bmatrix}

= A\mathbf{x}
$$


The Hoffman-Krusal Theorem states that the polyhedron $\{ \mathbf{x} \in \mathbb{R}^n : A\mathbf{x} \leq \mathbf{b}, \mathbf{x} \geq 0 \}$ is integral (meaning that all of its vertecies being integral and in our case the solution being integral) if the integer matrix A is totally unimodular and b being integral.


since b is a vector of integers, b is integral.

The requirments for a matrix being totally unmodular are the following

- entries $\in$ {-1,0,1}
- in each column , there are at most 2 non-zero entries
- rows can be partitioned into two sets s.t. each column:
    - if non-zero entries have opposite signs, they are in the same set
    - if non-zero entries have teh same signs, thay are in different sets

Since A meet all requirments A is totally unimodular.

Therefore the solution of this problem is integral.




# Question 2.

Recall the Minimum Weight Vertex Cover (VC) Problem: Given an undirected graph $G=(V, E)$, with node set $V$ and edge set $E$, where each node has a positive weight $w(v)$ associated with it (see the attached figure), the goal is to select a subset $V'\subseteq V$ of nodes such that every edge has at least one node incident to it, and the total selected node weight $\sum_{v\in V'} w(v)$ is minimized. 

* (4 points) Formulate the ILP for the VC problem for the attached graph, and solve it using **CVXPY** integer solver, for instance, `myVar = cp.Variable(<dim>, integer=True)`.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY** and comment on the relation between the two solutions.
* (2 points) Apply the rounding rule discussed in class to the optimal LP solution to obtain a solution to the ILP and compare it to the optimal ILP solution.

<!-- ![title](vertex_cover_example.png) -->


In a Minimum Weight Vertex Cover we want to minimize the total cost of picking vertices under the constraints that all edges must be connected to one vertex in the chosen set.

We can formulate this problem as follows.

$ \text{min} \sum_{v \in V'}{w(v)} \\$
s.t.
$\\ x_u + x_v \geq 1, (u,v) \in E \\$
$x =
\begin{cases}
1, & u \in V'\\
0, & \text{otherwise}
\end{cases}$

In [13]:
# Function returning Minimum Weight Vertex Cover (specifically for the attached graph).

def min_vc(x):
    c = [1, 3, 4, 2, 2, 4]

    # Objective function
    obj = cp.Minimize(c @ x)

    # Constraints
    A = np.array([[1, 1, 0, 0, 0, 0],
                [0, 1, 1, 0, 0, 0],
                [0, 1, 0, 0, 1, 0],
                [0, 0, 1, 1, 0, 0],
                [0, 0, 1, 0, 1, 0],
                [0, 0, 0, 1, 1, 0],
                [0, 0, 0, 0, 1, 1]])

    Ax = A @ x
    b = [1, 1, 1, 1, 1, 1, 1]

    constraints = [Ax >= b, x >= 0]

    # Solution
    prob = cp.Problem(obj, constraints)
    return prob.solve()

In [14]:
# ILP solution for the attached graph

import cvxpy as cp 
import numpy as np

# Variables
x = cp.Variable(6, integer=True)
opt = min_vc(x)

print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Objective value: 7.0
Solution vector: [-0.0, 1.0, -0.0, 1.0, 1.0, -0.0]


In [15]:
import cvxpy as cp 
import numpy as np

# Variables (no constraint on x being integral)
x = cp.Variable(6)
opt = min_vc(x)

print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Objective value: 7.0
Solution vector: [0.48, 0.52, 0.48, 0.52, 1.0, -0.0]


Without integer constraints on the variables they can be of any (positive) value. This solution does not make sense, because the node variables are supposed to be representations of booleans - either they are selected or not. We overcome this problem by rounding the nodes to 0 or 1:

$ \tilde{x}_u = 
\begin{cases}
1, & x_u^* \geq 0.5\\
0, & \text{otherwise}
\end{cases}
$

Then, we obtain the (relaxed) optimal solution by multiplying with nodes to the weights.

In this case, the optimal solution was identical to the ILP optimal, but this is not necessarily the case.

In [16]:
import cvxpy as cp 
import numpy as np

# Variables (no constraint on x being integral)
x = cp.Variable(6)
opt = min_vc(x)

# Apply the rounding rule
x_new = np.array([1 if float(n.value) >= 0.5 else 0 for n in x])

# Solve for updated x
c = [1, 3, 4, 2, 2, 4]
opt_new = c @ x_new
print(f"Objective value (after rounding): {round(opt_new, 2)}")
print(f"Solution vector (after rounding): {[round(float(i), 2) for i in x_new]}")

Objective value (after rounding): 7
Solution vector (after rounding): [0.0, 1.0, 0.0, 1.0, 1.0, 0.0]


# Question 3.

Consider a number of interpreters (Olof, Petra, Qamar,
  Rachel, Soren and Tao), as well as a set of languages (Arab,
  Bengali, Cantonese, Dutch, English, French and German). Each
  interpreter speaks a number of different languages (abbreviated by
  first letter), and has a certain per-diem integer cost:

\begin{array}{lll}
Interpreter & Languages & Cost\\
O & ABD & 3\\
P & C & 1\\
Q & CDG & 1\\
R & B & 2\\
S & G & 4\\
T & EF & 1\\
\end{array}

* (2 points) A *hypergraph* is a structure $H = (V,E)$ where $V$ is a set of vertices and $E$ is a collection of subsets of $V$. The special case when all subsets $e \in E$ have size exactly $2$ corresponds to the familiar case of a graph. A vertex cover in such a hypergraph is a subset $U \subseteq V$ such that $e \cap U \not = \emptyset$ for each $e \in E$ (note that this reduces to the usual vertex cover in graphs). Show that the problem of finding interpreters can be formulated as a vertex cover problem in a sutable hypergraph.
* (4 points) Develop a ILP formulation to finding the vertex cover of minimum cost in a hypergraph. The hypergraph can be represented as a $|V| \times |E|$ binary matrix $A$ where $A[i,j] = 1$ iff vertex $i$ is in edge $j$ and 0 otherwise. The costs for vertices are in an array $\texttt{c}$ where the cost of picking vertex $i$ is $c[i]$. Use the ILP formulation for the VC problem to hire the cheapest set of interpreters such that all languages are covered. Input the data above manuallly and solve it using **CVXPY**'s integer solver.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY**.
* (2 points) Explain why the two solutions above are same (different).


Note: The "problem of finding interpreters" is interpreted as finding a set of interpreters s.t. all langauges are represented.

Let $H=(V,E)$ where $V = \{O,P,Q,R,S,T\}$ and $E = \{(O), (O,R), (P,Q), (O,Q), (T), (T), (Q) \}$ where $e_i$ is the collection of interpreters speaking language $L_i$, $L = \{A,B,C,D,E,F,G\}$. A vertex cover in $H$ will select $U \subseteq V$ s.t. all languages are represented, thus solving the task.

In [42]:
import cvxpy as cp 
import numpy as np

# Variables
x = cp.Variable(6, integer=True)
c = [3, 1, 1, 2, 4, 1]
A = np.array([[1, 1, 0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 1],
              [0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 1, 0]])

# Objective function
obj = cp.Minimize(c @ x)

# Constraints
Ax = np.transpose(A) @ x
b = [1, 1, 1, 1, 1, 1, 1]

constraints = [Ax >= b, x >= 0]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Constraints: [Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ()))]
Constraints2: [Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Constant(CONSTANT, ZERO, ()))]
Objective value: 3.0
Solution vector: [1.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [45]:
import cvxpy as cp 
import numpy as np

# Variables
x = cp.Variable(6)
c = [3, 1, 1, 2, 4, 1]
A = np.array([[1, 1, 0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 1],
              [0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 1, 0]])

# Objective function
obj = cp.Minimize(c @ x)

# Constraints
Ax = np.transpose(A) @ x
b = [1, 1, 1, 1, 1, 1, 1]

constraints = [Ax >= b, x >= 0]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Objective value: 5.0
Solution vector: [1.0, 0.0, 1.0, -0.0, -0.0, 1.0]


<!-- * (2 points) Explain why the two solutions above are same (different). -->
The ILP solution equals the LP relaxation solution. However, this is not due to that fractions of vertices can not be part of an optimal solution, but rather due to the costs of the interpreters in the example. Since T is the only one speaking E and F, and O is the only one speaking A, we must choose those interpreters. Since O also speaks B and D, the remaining languages are C and G. Since Q speaks both C and G, and also has the lowest possible cost, there can not be any other optimal solution than choosing Q.

If we change the costs of Q to 2 and S to 1, we get the solution vector [1.0, 0.51, 0.49, 0.0, 0.51, 1.0], showing that the solution is integral because of the numbers in this actual example.

In mathematical terms, our constraints say that $x_1 \geq 1$ and $x_6 \geq 1$ (i.e. O and T is part of the solution). The constraints $x_1 + x_4 \geq 1$ and $x_1 + x_3 \geq 1$ is redundant since $x_1 \geq 1$ and $x_i \geq 0$ for all $x_i$ (a minimization based on only these constraints will set $x_3, x_4$ to zero). The remaining constraints are thus $x_2 + x_3 \geq 1$ and $x_3 + x_5 \geq 1$ where the objective function wants to minimize $x_2 + x_3 + 4 x_5$ which (regardless of ILP or LP) equals $x_3 = 1$. 

<!-- Since the objective function minimizes $\sum_{u \in V}{c_u x_u}$ and  -->


# Question 4. 
Consider the ILP and its LP relaxation corresponding to the VC problem for the graph $G$ given in the data file. This is a ***random graph*** $G(n,p)$ with $n=100$ vertices generated as follows: for each pair of vertices **independently**, we add an edge with probability $p=0.1$ (so the graph has about 1000 edges).

* **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.
* **b**. (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).
* **c**. (6 points) Consider the following rounding rule: we build up the vertex cover incrementally starting with $S:= \emptyset$. Now consider the edges in $G$ in any order. If an edge $(u,v)$ is already covered by a vertex in $S$, do nothing. Otherwise add to $S$ the vertex $u$ if $x^*(u) \geq x^*(v)$, or $v$ otherwise (where ${\bf x}^*$ is the LP optimum solution computed in part (b).  Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b). Compare the cost of the solution produced by this rule to the optimal solution.

In [26]:
# Optimal solution (ILP)

import cvxpy as cp 

# Variables
x = cp.Variable(100, integer=True)

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

# Create list of edges from graph
edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))

# Objective function
obj = cp.Minimize(sum(x))

# Constraints
constraints = [x >= 0]

for (m,n) in edges:
    constraints.append(x[m] + x[n] >= 1)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt, 2)}")
print(f"Solution vector: {[round(float(i.value), 2) for i in x]}")

Objective value: 70.0
Solution vector: [1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]


In [27]:
# LP relaxation

import cvxpy as cp 

# Variables
x = cp.Variable(100)

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))

# Objective function
obj = cp.Minimize(sum(x))

# Constraints
constraints = [x >= 0]

for (m,n) in edges:
    constraints.append(x[m] + x[n] >= 1)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value (before rounding): {round(opt, 2)}")
node_values = [(float(n.value)) for n in x]
print(f"Solution vector (before rounding): {node_values}")

rounded_nodes = [1 if n >= 0.499999 else 0 for n in node_values]
print(f"Objective value (after rounding): {sum(rounded_nodes)}")
print(f"Solution vector (after rounding): {rounded_nodes}")

Objective value (before rounding): 50.0
Solution vector (before rounding): [0.500000000027591, 0.49999999999986944, 0.5000000000044335, 0.49999999999428957, 0.5000000000030659, 0.5000000000068782, 0.5000000000071365, 0.5000000000034065, 0.49999999999804845, 0.49999999999884565, 0.49999999998820593, 0.49999999999593014, 0.5000000000206385, 0.49999999999652, 0.49999999999231026, 0.5000000000046911, 0.5000000000021066, 0.49999999999576794, 0.4999999999753837, 0.5000000000007743, 0.500000000018824, 0.49999999999131567, 0.5000000000028995, 0.4999999999978037, 0.4999999999993328, 0.499999999995314, 0.5000000000033956, 0.5000000000066869, 0.5000000000020646, 0.5000000000053395, 0.5000000000242643, 0.4999999999894045, 0.49999999999095024, 0.5000000000024466, 0.5000000000105581, 0.5000000000040499, 0.4999999999942369, 0.5000000000039171, 0.5000000000016301, 0.500000000003655, 0.49999999999829114, 0.5000000000174603, 0.5000000000045337, 0.500000000001898, 0.5000000000172209, 0.4999999999968136, 

The optimal value of the LP relxation before rounding is 50 (by simply assigning each variable 0.5). Since vertices must be integral, we apply the rounding rule (here, because of rounding errors in the LP solver, we assume values $v \geq 0.49999 = 1$), which results in all vertices being picked and the objective value equal 100.

In [11]:
import cvxpy as cp 

# Variables
x = cp.Variable(100)

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))

# Objective function
obj = cp.Minimize(sum(x))

# Constraints
constraints = [x >= 0]

for (m,n) in edges:
    constraints.append(x[m] + x[n] >= 1)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
node_values = [(float(n.value)) for n in x]
rounded_nodes = [1 if n >= 0.49999 else 0 for n in node_values]
print(f"Solution vector: {rounded_nodes}")

# Create vertex cover S
S = []
for (i,j) in edges:
    if i in S or j in S:
        pass
    else:
        if (rounded_nodes[i]) >= (rounded_nodes[j]):
            S.append(i)
        else:
            S.append(j)

print(f"Vertex cover: {S}")

# Update optimal value
new_opt = len(S)
vec = [0] * 100
for i in S:
    vec[i] = 1

print(f"Objective value: {round(new_opt, 2)}")
print(f"Solution vector: {vec}")

# Check if solution set of nodes is a vertex cover
print("Checking if solution is a vertex cover...")
cover = [i for i, val in enumerate(rounded_nodes) if val == 1]
print(f"Solution vertex cover indices: {cover}")
for (i,j) in edges:
    if i in cover or j in cover:
        pass
    else:
        print("The solution is not a vertex cover")
        break
print("The solution is a vertex cover!")

Solution vector: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Vertex cover: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95, 97]
Objective value: 93
Solution vector: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

<!-- Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b).  -->
This algorithm creates a valid vertex cover since it iterates through the edges $e_i = (u,v) $ and checks whether $u \in S$ or $v \in S$ and if not, adds either vertix to $S$ (based on the weight computed in the LP). In our graph, all vertices have the same weight $(0.5)$, and therefore $u$ will be added to $S$ in each iteration. Since this check is done on all edges we know that the algorithm will produce a valid vertex cover. Furthermore, if $u = 0.5$ and $v = 0.5$, it will not add both $u, v$ to $S$, but instead only add $u$ and continue with the iteration. Therefore, it will never have a larger cost than that of the LP relaxation solution in (b). In fact, it will have a lower cost, since we know from (a) that the optimal number of vertices is $70$, thus there will be comparisons of edges where one or both vertices are included in $S$.

<!-- Compare the cost of the solution produced by this rule to the optimal solution. -->
The cost of the optimal solution is $70$ compared to $93$ in this solution. It is much worse than the optimal solution, but slightly better than the LP relaxation solution in (b). Since there are no "intelligent" iteration through the list of edges, the alhorithm adds vertices continuously while iterating through the edges. Some of these vertices will be redundant to the optimal vertex cover (if it is not the case that specifically the first 70 vertices is part of the vertex cover). Therefore, the number of vertices is much higher than in the optimal solution.

For example, by simply adding $v$ insatead of $u$ in each iteration, a better vertex cover (77 nodes) will be created without any optimization or comparisons of nodes. Adding $u$ or $v$ randomly will produce a solution of about 80-90 vertices. This points to the fact that the order of edges are of most importance when utilizing this method. See below for a visualization.

In [ ]:
import random

with open("graph.txt", "r") as f:
    graph = [[int(num) for num in line.split()] for line in f]

edges = []
for i in range(len(graph)):
    for j in range(len(graph[i])):
        if graph[i][j] == 1:
            edges.append((i,j))
            
# Create vertex cover S
S = []
for (i,j) in edges:
    if i in S or j in S:
        pass
    else:
        S.append(j)

# S_random picks u/v randomly
S_random = []
for (i,j) in edges:
    if i in S_random or j in S_random:
        pass
    else:
        S_random.append(random.choice([i, j]))

opt = len(S)
opt_random = len(S_random)

print(f"Vertex cover size (picking v from (u,v)): {opt}")
print(f"Vertex cover size (picking randomly from (u,v)): {opt_random}")

# Check if solution set of nodes is a vertex cover
print("Checking if solution is a vertex cover...")
cover = [i for i, val in enumerate(vec) if val == 1]
print(f"Solution vertex cover indices: {cover}")
for (i,j) in edges:
    if i in cover or j in cover:
        pass
    else:
        print("The solution is not a vertex cover")
        break
print("The solution is a vertex cover!")

Vertex cover size (picking v from (u,v)): 77
Vertex cover size (picking randomly from (u,v)): 86
Checking if solution is a vertex cover...
Solution vertex cover indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95, 97]
The solution is a vertex cover!
